# Customize output format in prompt

## Load model

In [1]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-search-preview")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff4358a0f0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff435a8590>, root_client=<openai.OpenAI object at 0xffff43b77c20>, root_async_client=<openai.AsyncOpenAI object at 0xffff4358a150>, model_name='gpt-4o-search-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))

## Load prompt

In [2]:
from langchain_core.prompts import load_prompt
prompt = load_prompt(path='../prompts/ES.json')

prompt

PromptTemplate(input_variables=['FREQUENCY', 'TABLE', 'TICKER'], input_types={}, partial_variables={}, template='La siguiente tabla muestra los retornos del activo financiero con ticker {TICKER}, calculados como la variación porcentual entre el precio de cierre de cada fecha y el de la fecha inmediatamente anterior registrada en la tabla. La frecuencia aplicada al cálculo ha sido definida como {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nTABLA:\n\n{TABLE}\n\nExplica los motivos detrás de los valores más extremos (tanto positivos como negativos), utilizando noticias relevantes que puedan justificar dichos movimientos. Para cada valor, proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')

## Custom Output Parser

### Create Pydantic model

In [3]:
from pydantic import BaseModel, Field

class News(BaseModel):
    stock: str = Field(description="stock symbol")
    date: str = Field(description="date of the news")
    value: float = Field(description="significant acumulated return value of the stock")
    title: str = Field(description="title of the news")
    url: str = Field(description="url of the news")
    source: str = Field(description="source of the news")
    explanation: str = Field(description="explanation of the news relevance to the stock")
    
class NewsList(BaseModel):
    news: list[News] = Field(description="list of news")

### Associate Pydantic model with parser

In [4]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=NewsList)

parser

PydanticOutputParser(pydantic_object=<class '__main__.NewsList'>)

### Add parser to prompt

In [5]:
prompt.template = prompt.template + '\n\n{format_instructions}\n'
prompt.template 

'La siguiente tabla muestra los retornos del activo financiero con ticker {TICKER}, calculados como la variación porcentual entre el precio de cierre de cada fecha y el de la fecha inmediatamente anterior registrada en la tabla. La frecuencia aplicada al cálculo ha sido definida como {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nTABLA:\n\n{TABLE}\n\nExplica los motivos detrás de los valores más extremos (tanto positivos como negativos), utilizando noticias relevantes que puedan justificar dichos movimientos. Para cada valor, proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.\n\n{format_instructions}\n'

In [6]:
prompt.partial_variables.update({"format_instructions": parser.get_format_instructions()})
prompt

PromptTemplate(input_variables=['FREQUENCY', 'TABLE', 'TICKER'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"News": {"properties": {"stock": {"description": "stock symbol", "title": "Stock", "type": "string"}, "date": {"description": "date of the news", "title": "Date", "type": "string"}, "value": {"description": "significant acumulated return value of the stock", "title": "Value", "type": "number"}, "title": {"description": "title of the news", "title": "Title", "type": "string"}, "url": {"description": "url 

## Chain

### Define chain

In [7]:
chain = prompt | model | parser
chain

PromptTemplate(input_variables=['FREQUENCY', 'TABLE', 'TICKER'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"News": {"properties": {"stock": {"description": "stock symbol", "title": "Stock", "type": "string"}, "date": {"description": "date of the news", "title": "Date", "type": "string"}, "value": {"description": "significant acumulated return value of the stock", "title": "Value", "type": "number"}, "title": {"description": "title of the news", "title": "Title", "type": "string"}, "url": {"description": "url 

### Preprocess input data

In [8]:
ticker = 'AAPL'
freq = 'D'

In [ ]:
import utils

df = utils.calculate_significant_returns_simple(
    ticker=ticker,
    start='2024-01-01',
    end='2024-12-31',
    freq='D',
    n=5
)

df

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,AAPL
Date,
2024-06-11,7.264912
2024-05-03,5.981623
2024-08-05,-4.816698
2024-04-11,4.327092
2024-03-21,-4.085749


### Invoke chain

In [10]:
chain

PromptTemplate(input_variables=['FREQUENCY', 'TABLE', 'TICKER'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"News": {"properties": {"stock": {"description": "stock symbol", "title": "Stock", "type": "string"}, "date": {"description": "date of the news", "title": "Date", "type": "string"}, "value": {"description": "significant acumulated return value of the stock", "title": "Value", "type": "number"}, "title": {"description": "title of the news", "title": "Title", "type": "string"}, "url": {"description": "url 

In [11]:
chain.get_prompts()

[PromptTemplate(input_variables=['FREQUENCY', 'TABLE', 'TICKER'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"News": {"properties": {"stock": {"description": "stock symbol", "title": "Stock", "type": "string"}, "date": {"description": "date of the news", "title": "Date", "type": "string"}, "value": {"description": "significant acumulated return value of the stock", "title": "Value", "type": "number"}, "title": {"description": "title of the news", "title": "Title", "type": "string"}, "url": {"description": "url

In [12]:
output = chain.invoke({"TABLE": df, "TICKER": ticker, "FREQUENCY": freq})

In [13]:
data = output.model_dump()
data

{'news': [{'stock': 'AAPL',
   'date': '2024-06-11',
   'value': 7.264912,
   'title': "Apple's Stock Price Reaches All-Time High After WWDC Announcements",
   'url': 'https://www.macrumors.com/2024/06/11/aapl-all-time-high-wwdc-2024/',
   'source': 'MacRumors',
   'explanation': "El 11 de junio de 2024, las acciones de Apple subieron un 7.26% alcanzando un máximo histórico tras el anuncio de 'Apple Intelligence', una suite de funciones de IA integradas en iOS 18, iPadOS 18 y macOS Sequoia, durante la WWDC. Esta integración de IA generativa y la asociación con OpenAI para mejorar la experiencia del usuario generaron optimismo entre los inversores sobre un nuevo ciclo de actualización de iPhones."},
  {'stock': 'AAPL',
   'date': '2024-08-05',
   'value': -4.816698,
   'title': 'Apple Stock Declines Amid Concerns Over AI Integration Challenges',
   'url': 'https://www.investorplace.com/2024/06/apple-stock-hits-all-time-highs-on-ai-news-time-to-take-profits/',
   'source': 'InvestorPlace

In [ ]:
import pandas as pd
pd.DataFrame(data['news']).style

,stock,date,value,title,url,source,explanation
0,AAPL,2024-01-03,-4.000000,"Apple pierde 100.000 millones en bolsa en un solo día, más que el valor de Airbnb o Iberdrola",https://www.businessinsider.es/economia/apple-empieza-mal-2024-pierde-100000-millones-bolsa-solo-dia-1353976,Business Insider España,"El 3 de enero de 2024, las acciones de Apple cayeron un 4% después de que Barclays rebajara su calificación a 'infraponderar', citando ventas mediocres del iPhone 15 en China y una falta de recuperación en las ventas de Macs, iPads y wearables. Esta caída redujo la capitalización bursátil de Apple en más de 100.000 millones de dólares."
1,AAPL,2024-03-21,-3.700000,Las acciones de Apple cayeron un 4% después de la demanda de Estados Unidos a la compañía por monopolio,https://www.infobae.com/estados-unidos/2024/03/22/las-acciones-de-apple-cayeron-un-37-despues-de-la-demanda-de-estados-unidos-a-la-compania-por-monopolio/,Infobae,"El 21 de marzo de 2024, el Departamento de Justicia de Estados Unidos presentó una demanda contra Apple, alegando que la compañía había creado un monopolio en el mercado de los teléfonos inteligentes con su modelo iPhone. Tras el anuncio, las acciones de Apple cayeron un 3,7%."
